In [1]:
import requests
import pandas as pd
from funcoes import *

In [ ]:
r = requests.get("https://mof.tech.northwestern.edu/mofs.json")

print(r)

In [ ]:
data = r.json()['results']

In [ ]:
df = pd.DataFrame.from_dict(data)

In [ ]:
df = funcao_extrair_features_isotherms(df)

In [ ]:
def importar_dados(dataframe):
    'importar os dados provenientes do site das MOFs'
    dados_hmof = pd.read_excel(dataframe)
    return dados_hmof

In [ ]:
dataframe = ('Pasta1.xlsx')
df_import = importar_dados(dataframe)

## Importações



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Divisão treino-teste

In [ ]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
FEATURES = ['lcd', 'pld', 'void_fraction', 'surface_area_m2g', 'surface_area_m2cm3', 'CO2_0.01', 'CO2_0.1', 'CO2_2.5', 'CO2_0.05', 'CO2_0.5',
            'N2_0.09', 'N2_0.9', 'CH4_2.5', 'CH4_4.5', 'CH4_0.05', 'CH4_0.5', 'CH4_0.9', 'CH4_35', 'H2_2', 'H2_100']
TARGET = ['Xe_1']

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

In [ ]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

In [ ]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

In [ ]:
print(X_treino)
print()
print(y_treino)

In [ ]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

In [ ]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

In [ ]:
for p in minha_MLP.parameters():
    print(p)

In [ ]:
y_prev = minha_MLP(X_treino)
y_prev

In [ ]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

In [ ]:
minha_MLP.train()

In [ ]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

In [ ]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

In [ ]:
minha_MLP.eval()

In [ ]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

In [ ]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')